In [1]:
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, *-*"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, *-*"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, *-*"

Installed Packages Microsoft.SemanticKernel.Agents.Abstractions, 1.32.0-alpha Microsoft.SemanticKernel.Agents.Core, 1.32.0-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.32.0-alpha Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.32.0

In [2]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;

In [3]:
var deployment = "gpt-4o-mini";
var endpoint = "Your AOAI endpoint";
var key = "Your AOAI key";

In [4]:
#pragma warning disable SKEXP0010

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deployment, endpoint, key)
    .Build();

In [5]:
#r "nuget: Azure.AI.Projects, 1.0.0-beta.2"

#r "nuget: Azure.Identity, 1.13.1"

Installed Packages Azure.AI.Projects, 1.0.0-beta.2 Azure.Identity, 1.13.1

In [6]:
using System;
using System.Collections.Generic;
using System.Threading.Tasks;

using Azure.Identity;
using Azure.AI.Projects;
using Azure.Core;
using Azure.Core.Pipeline;

In [7]:
internal class CustomHeadersPolicy : HttpPipelineSynchronousPolicy
{
    public override void OnSendingRequest(HttpMessage message)
    {
        message.Request.Headers.Add("x-ms-enable-preview", "true");
    }
}

In [8]:
public sealed class SearchPlugin
{
        [KernelFunction, Description("Search by Bing")]
        [System.Diagnostics.CodeAnalysis.SuppressMessage("Design", "CA1024:Use properties where appropriate", Justification = "Too smart")]
        public async Task<string> Search([Description("search Item")]
            string searchItem)
        {
            var connectionString = "Your Azure AI Agent Service Connection String";
            var clientOptions = new AIProjectClientOptions();

            // Adding the custom headers policy
            clientOptions.AddPolicy(new CustomHeadersPolicy(), HttpPipelinePosition.PerCall);
            var projectClient = new AIProjectClient(connectionString, new DefaultAzureCredential(), clientOptions);

            ConnectionResponse bingConnection = await projectClient.GetConnectionsClient().GetConnectionAsync("kinfey-bing-search");
            var connectionId = bingConnection.Id;

            AgentsClient agentClient = projectClient.GetAgentsClient();

            ToolConnectionList connectionList = new ToolConnectionList
            {
                ConnectionList = { new ToolConnection(connectionId) }
            };
            BingGroundingToolDefinition bingGroundingTool = new BingGroundingToolDefinition(connectionList);

            Azure.Response<Azure.AI.Projects.Agent> agentResponse = await agentClient.CreateAgentAsync(
            model: "gpt-4-1106-preview",
            name: "my-assistant",
            instructions: "You are a helpful assistant.",
            tools: new List<ToolDefinition> { bingGroundingTool });
            Azure.AI.Projects.Agent agent = agentResponse.Value;

            // Create thread for communication
            Azure.Response<AgentThread> threadResponse = await agentClient.CreateThreadAsync();
            AgentThread thread = threadResponse.Value;

            // Create message to thread
            Azure.Response<ThreadMessage> messageResponse = await agentClient.CreateMessageAsync(
                thread.Id,
                MessageRole.User,
                "How does wikipedia explain Euler's Identity?");
            ThreadMessage message = messageResponse.Value;

            // Run the agent
            Azure.Response<ThreadRun> runResponse = await agentClient.CreateRunAsync(thread, agent);

            do
            {
                await Task.Delay(TimeSpan.FromMilliseconds(500));
                runResponse = await agentClient.GetRunAsync(thread.Id, runResponse.Value.Id);
            }
            while (runResponse.Value.Status == RunStatus.Queued
                || runResponse.Value.Status == RunStatus.InProgress);

            Azure.Response<PageableList<ThreadMessage>> afterRunMessagesResponse
                = await agentClient.GetMessagesAsync(thread.Id);
            IReadOnlyList<ThreadMessage> messages = afterRunMessagesResponse.Value.Data;

            string searchResult = "";

            // Note: messages iterate from newest to oldest, with the messages[0] being the most recent
            foreach (ThreadMessage threadMessage in messages)
            {
                Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");

                if(threadMessage.Role.ToString().ToLower()=="assistant")
                {
                    foreach (MessageContent contentItem in threadMessage.ContentItems)
                    {
                        if (contentItem is MessageTextContent textItem)
                        {
                            Console.Write(textItem.Text);
                            searchResult = textItem.Text;
                        }
                        break;
                        // Console.WriteLine();
                    }
                }
            }

            return searchResult;
        }
}

In [9]:
public sealed class SavePlugin
{
        [KernelFunction, Description("Save blog")]
        [System.Diagnostics.CodeAnalysis.SuppressMessage("Design", "CA1024:Use properties where appropriate", Justification = "Too smart")]
        public async Task<string> Save([Description("save blog content")]
            string content)
        {
            Console.Write("###"+content);
            var connectionString = "eastus.api.azureml.ms;6415ebd4-1dd7-430f-bd4d-2f5e9419c1cd;rg-kinfey-ai;kinfey-agent-project-23ra";
            AgentsClient client = new AgentsClient(connectionString, new DefaultAzureCredential());
            Azure.Response<Azure.AI.Projects.Agent> agentResponse = await client.CreateAgentAsync(
                model: "gpt-4o-mini",
                name: "code-agent",
                instructions: "You are a personal python assistant. Write and run code to answer questions.",
                tools: new List<ToolDefinition> { new CodeInterpreterToolDefinition() });
            Azure.AI.Projects.Agent agent = agentResponse.Value;
            Azure.Response<PageableList<Azure.AI.Projects.Agent>> agentListResponse = await client.GetAgentsAsync();
            Azure.Response<AgentThread> threadResponse = await client.CreateThreadAsync();
            AgentThread thread = threadResponse.Value;
             Azure.Response<ThreadMessage> messageResponse = await client.CreateMessageAsync(
            thread.Id,
            MessageRole.User,
            @"You are my Python programming assistant. Generate code and execute it according to the following requirements
                1. Save" + content +     @"file as blog-{YYMMDDHHMMSS}.md
                2. give me the download this file link
            ");
            ThreadMessage message = messageResponse.Value;
            Azure.Response<PageableList<ThreadMessage>> messagesListResponse = await client.GetMessagesAsync(thread.Id);
            Azure.Response<ThreadRun> runResponse = await client.CreateRunAsync(
            thread.Id,
            agent.Id);
            ThreadRun run = runResponse.Value;
            do
            {
                        await Task.Delay(TimeSpan.FromMilliseconds(500));
                        runResponse = await client.GetRunAsync(thread.Id, runResponse.Value.Id);
            }
            while (runResponse.Value.Status == RunStatus.Queued
                        || runResponse.Value.Status == RunStatus.InProgress);
            Azure.Response<PageableList<ThreadMessage>> afterRunMessagesResponse = await client.GetMessagesAsync(thread.Id);
            IReadOnlyList<ThreadMessage> messages = afterRunMessagesResponse.Value.Data;

            foreach (ThreadMessage threadMessage in messages)
            {
                        // Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
                        foreach (MessageContent contentItem in threadMessage.ContentItems)
                        {
                            
                            if (contentItem is MessageTextContent textItem)
                            {
                                // Console.Write(textItem.Text);

                                if(textItem.Annotations is not null && textItem.Annotations.Count > 0)
                                {

                                    if(textItem.Annotations[0] is MessageTextFilePathAnnotation pathItem )
                                    {
                                        // Console.Write($"[Download file]({pathItem.Text})");

                                        Azure.Response<AgentFile> agentfile = await client.GetFileAsync(pathItem.FileId);

                                        Azure.Response<BinaryData> fileBytes = await client.GetFileContentAsync(pathItem.FileId);

                                        Console.Write(agentfile.Value.Filename);

                                        var mdfile =System.IO.Path.GetFileName(agentfile.Value.Filename);

                                        using System.IO.FileStream stream = System.IO.File.OpenWrite($"./blog/{mdfile}");
                                        fileBytes.Value.ToStream().CopyTo(stream);

                                    }

                                }

                            }
                            // Console.WriteLine();
                        }


            }
        
            return "Saved";
        }
}

In [10]:
const string SearchHostName = "Search";
const string SearchHostInstructions = "You are a search expert, help me use tools to find relevant knowledge";

In [11]:
#pragma warning disable SKEXP0110

ChatCompletionAgent search_agent =
            new()
            {
                Name = SearchHostName,
                Instructions = SearchHostInstructions,
                Kernel = kernel,
                Arguments = new KernelArguments(new AzureOpenAIPromptExecutionSettings() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() }),
            };

In [12]:
const string SaveHostName = "SaveBlog";
const string SavehHostInstructions = "Save blog content. Respond with 'Saved' to when your blog are saved.";

In [13]:
#pragma warning disable SKEXP0110

ChatCompletionAgent save_blog_agent =
            new()
            {
                Name = SaveHostName,
                Instructions = SavehHostInstructions,
                Kernel = kernel,
                Arguments = new KernelArguments(new AzureOpenAIPromptExecutionSettings() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() }),
            };

In [14]:
private const string WriteBlogName = "WriteBlog";
private const string WriteBlogInstructions =
        """
        You are a blog writer, please help me write a blog based on bing search content.
        """;

In [15]:
#pragma warning disable SKEXP0110

ChatCompletionAgent write_blog_agent =
            new()
            {
                Name = WriteBlogName,
                Instructions = WriteBlogInstructions,
                Kernel = kernel
            };

In [16]:
#pragma warning disable SKEXP0110

KernelPlugin search_plugin = KernelPluginFactory.CreateFromType<SearchPlugin>();
search_agent.Kernel.Plugins.Add(search_plugin);

In [17]:
#pragma warning disable SKEXP0110

KernelPlugin save_blog_plugin = KernelPluginFactory.CreateFromType<SavePlugin>();
save_blog_agent.Kernel.Plugins.Add(save_blog_plugin);

In [18]:
 #pragma warning disable SKEXP0110   
 using System.Threading;
    


private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
        // Terminate when the final message contains the term "approve"
        protected override Task<bool> ShouldAgentTerminateAsync(Microsoft.SemanticKernel.Agents.Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
            => Task.FromResult(history[history.Count - 1].Content?.Contains("Saved", StringComparison.OrdinalIgnoreCase) ?? false);
}

In [19]:
 #pragma warning disable SKEXP0110

AgentGroupChat chat =
            new(search_agent, write_blog_agent,save_blog_agent)
            {
                ExecutionSettings =
                    new()
                    {
                        TerminationStrategy =
                            new ApprovalTerminationStrategy()
                            {
                                // Only the art-director may approve.
                                Agents = [save_blog_agent],
                                // Limit total number of turns
                                MaximumIterations = 10,
                            }
                    }
            };

In [20]:
 #pragma warning disable SKEXP0110

ChatMessageContent input = new(AuthorRole.User, """
                    I am writing a blog about GraphRAG. Search for the following 2 questions and write a Chinese blog based on the search results ,save it           
                        1. What is Microsoft GraphRAG?
                        2. Vector-based RAG vs GraphRAG
                    """);
chat.AddChatMessage(input);

In [21]:
#pragma warning disable SKEXP0110   
#pragma warning disable SKEXP0001
      


await foreach (ChatMessageContent content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}

# Assistant - Search: 'Currently, I am unable to retrieve the necessary information to answer your questions about Microsoft GraphRAG and its comparison with vector-based RAG systems due to technical difficulties. However, I can help you draft a potential structure for your blog based on general knowledge, and once I can perform searches, I can provide updates. Here's a preliminary outline:

---

### 什么是 Microsoft GraphRAG?

**引言**  
Microsoft GraphRAG是Microsoft推出的一种先进的生成式人工智能框架。它结合了图数据库技术与传统的检索增强生成（RAG）模型，以提供更高效的信息检索和内容生成能力。

**核心功能**  
- **图数据库集成**：利用图数据库的强大结构，GraphRAG能够更好地理解和处理复杂的关系和数据。
- **智能检索**：通过对关系数据的智能分析，GraphRAG能迅速找到最相关的信息片段。
- **生成技术**：结合生成式模型，GraphRAG不仅能检索信息，还能生成高质量的文字内容。

### Vector-based RAG 与 GraphRAG 的对比

**概述**  
向量基础的RAG模型主要依赖文本的向量表示和距离计算来进行检索，而GraphRAG则采用更为复杂的图结构进行信息处理。

**技术比较**  
- **信息检索效率**：GraphRAG在处理关系密集的数据时表现优异，能够提供更相关的查询结果。
- **数据结构**：向量模型通常使用固定维度的向量空间，而GraphRAG则利用图节点和边的灵活性，更直观地表示数据之间的关系。
- **使用场景**：GraphRAG适合那些有复杂关系的数据集，比如社交网络分析或推荐系统，而向量模型则适合直接基于文本的问答系统。

**